# Counterfactual GAN for macroeconomic policy simulations

The core idea is to use **counterfactual shifts** to simulate macroeconomic scenarios based on changes in one key indicator variable $i$ - the conditioning variable $c$. That is to say, to determine the effectiveness of for example a central bank policy measured by $\textbf{x} = {x_1, x_2, ..., x_m}, \textbf{x} \subset X \land i \in X$ (with $\textbf{x} = X \setminus y$) when the exchange rate for a certain currency would $i$ would have been different from that found in the real data (counterfactually shifting $i$ to $c$ under macroeconomic scenario $s$), one needs to be able to model the effect of changes in variable $i$ on all other variables $\textbf{x}$ in the dataset $X$.

##### The Bayesian model

To formalize, the generative model needs to learn both the joint probability

$$ P(i_t,\textbf{x}_t) = P(X_{t-n:t-1}|i_{t-n:t-1},\textbf{x}_{t-n:t-1}), i \in X, \textbf{x} \subset X, i \notin \textbf{x}$$

of the conditioning variable $i$ and the conditioned variables $\textbf{x} \subset X$ at time $t$ conditional on previous timesteps ${t-n, ..., t-1}$, **and** the conditional probability of $\textbf{x}$ given $i$ at time $t$ conditioned on a macroeconomic scenario $s$, where $i_t^s = c_t$,

$$P(\textbf{x}_t|c_t)$$

From a Bayesian point of view, it makes sense to take the distribution in the real data as the prior. This means that the CGAN needs to
1. Approximate the $n$ historical autoregressive priors for all variables $p(X_t|X_{t-n:t-1})$ at $X_t$ given $X_{t-n}, ..., X_{t-1}$.
2. Infer the posterior distribution $p(X_{t-n:t-1},\beta,\gamma|X_t) \propto p(X_t|X_{t-n:t-1},\beta,\gamma)p(X_t|X_{t-n:t-1})$, where $\gamma$ maps contemporary relationships between variables at time $t$ and $\beta$ models the autoregressive relationship over time for all variables, mapping $X_{t-n:t-1}$ on to $X_t$.
3. Use the inferred posterior of the variables $\textbf{x}$ to generate $Y = c,P(\textbf{x}|c,s,\beta,\gamma)$, where $c$ is the conditioned form of $i$ shifted to simulate some explicitly programmed counterfactual scenario $s$, and $P(\textbf{x}|c,s)$ are the values of $\textbf{x}$ mapped onto this counterfactual scenario $s$. 

Given that a scenario $s$ can also be modeled as a regime, it might be good to look into RL research on the topic of policy switches in model-based RL.

##### Counterfactuals

From a causal inference standpoint, policy simulations should result in an estimate of either individual treatment effects ($ITE_i = Y_1^{(i)} - Y_0^{(i)}$ for treatment (1) and non-treatment (0) conditions on unit $i$), or average treatment effects ($ATE = 1/N \sum\limits_i y_1(i) - y_0(i)$) of the policy intervention $c$ on the other variables $\textbf{x}$.

In the language of causal analysis, the factual variable $i$ and counterfactual variable $c$ under scenario $s$ are investigated. Therefore the generator should learn to model not only the generated variables but also underlying confounders, which are hypothesized to aide in generating more realistic simulations.

##### Training and inference

Translating the Bayesian model to a GAN framework,
* The generator generates datapoints conditional both on $\textbf{A}$) autoregressive previous timesteps ${t-n, ..., t-1}$ and $\textbf{B}$) lateral relationships between variables (longitudinal and contemporary effects), ignoring $\textbf{C}$) the conditional variable $c$ during training.
* The discriminator is used to train the generator to model $\textbf{A}$) and $\textbf{B}$) as realistically as possible.
* During inference $c$ is shifted to simulate the counterfactual condition $\textbf{C}$), letting the generator model the effect of $c$ on the $x \subset X$ variables of interest.

The training data will come from a training pool consisting of datapoints for some 100 or so countries, shuffled for training purposes but with country encodings to allow the model to learn different distributions. The validation data consists of a separate pool of 15 heterogeneous countries.

The model learns to predict variables $\textbf{i}_t = \textbf{x}_t$ at time $t$ given $\textbf{x}_{t-1}$ within the context of $\textbf{x}_{t-n:t-1}$. For practical purposes the data feed consists of multivariate sequences of length $n-1$, and multivariate labels with length $1$. During inference, the model needs to have access to predictions at it made at previous timesteps in order to set the generated context $\hat{\textbf{x}}_{t-n:t-1}$.

##### Loss functions

The correctness of the generator can be measured by proxy by looking at **reconstruction losses** from historical data. I.e., provide it with a prompt of $m$ sequences on unseen historical country data, and determine how close it matched the real data for the remaining $n-m$ observations. Given the natural differences in country data, it might be good to take a pool of representative countries as the validation dataset. This will also help mitigate covariate shift issues (c.f. [Johansson et al. 2018](https://arxiv.org/abs/1605.03661)).

Since we are dealing with counterfactuals, the loss metric cannot be measured directly from labeled data. However, a proxy of **counterfactual loss** could be to measure the proportional, variable-bound divergence of the simulated data from the real distribution, for example through Kullback-Leibler divergence, Fisher information, or optimal transport. The assumption underlying such an approach would be that of *real-world constraints* on macroeconomic data, i.e. the overall quantities of the indicator variable won't change as a result of counterfactual evidence, but their underlying distribution might.

##### Counterfactual GAN

The generator models $\beta$ and $\gamma$ through the same parameter set, as is current best practice in sequence modeling. One obvious candidate architecture that springs to mind are transformer networks, for example the [universal transformer](https://arxiv.org/abs/1807.03819) architecture. 

A correct measurement of the loss will need to incorporate covariate shifts across country datasets into the function. One widely-used metric to include a measure thereof in the loss function is maximum-mean discrepancy (MMD, c.f. [Gretton et al., 2012](http://www.jmlr.org/papers/volume13/gretton12a/gretton12a.pdf), [Sutherland et al., 2107](https://arxiv.org/abs/1611.04488)), which can be estimated empirically over the means of the real and simulated data distributions. 

*// TODO: investigate loss functions for time series generation and simulation models.*

##### Simulation

*// TODO: implement inference layer that allows for the combination of manually conditioned timeseries data $c$ and generated data $x$.*


###### Generalization

*// TODO: develop tests and visualisations to check the generalization capabilities of the model.*

In [ ]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
from tensorflow import keras

%matplotlib inline

In [ ]:
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:20,.4f}'.format

In [ ]:
sns.set_style("whitegrid")
sns.set_palette("colorblind")

In [ ]:
tf.__version__

#### Load and prepare the data

In [ ]:
# validation_countres = ['France']

In [ ]:
country = 'France'

In [ ]:
features_df = pd.read_csv('features/m_one/%s_features.csv' % country, sep=';', header=0)
labels_df = pd.read_csv('features/m_one/labels_interpolated.csv', sep=';', header=0)

In [ ]:
combined_df = pd.concat([features_df, labels_df.drop(columns=['date'])], axis=1)

In [ ]:
combined_df.head()

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
[sns.lineplot(x='date', y=c, markers=True, ax=ax, label=c, data=combined_df) for c in list([country, 'lending interest rate', 'real interest rate', 'inflation', 'gross domestic savings', 'government debt service'])]

xticks=ax.xaxis.get_major_ticks()
for i in range(len(xticks)):
    if i % 12 == 1:
        xticks[i].set_visible(True)
    else:
        xticks[i].set_visible(False)

ax.set_xticklabels(combined_df['date'], rotation=45);

In [ ]:
combined_df.columns

In [ ]:
base_feature_df = combined_df[['date', 'bank capital to assets ratio', 'bank nonperforming loans', 'cereal yield',
                               'energy imports', 'food exports', 'high-tech exports', 'inflation',
                               'lending interest rate', 'life expectancy', 'population density', 'real interest rate',
                               'broad money', 'exports of goods and services', 'gross domestic savings',
                               'high-tech value added', 'household consumption expenditure',
                               'imports of goods and services', 'listed companies', 'manufacturing value added',
                               'r and d spend', 'services trade', 'trade', 'government debt service',
                               'government interest payments external debt', 'government tax revenue', 'birth deaths',
                               'broadband subscriptions', 'electricity access', 'co2 emissions',
                               'electricity consumption', 'mobile subscriptions', 'newborns', 'overweight',
                               'rural population', 'urban population', country]]

In [ ]:
base_feature_df['label'] = base_feature_df[country].shift(periods=1)
base_df = base_feature_df.drop(country, axis=1).fillna(0.00);
base_df.set_index('date');

In [ ]:
sim_df = base_df[['inflation']]
sim_df.shape

In [ ]:
num_obs = len(sim_df)
num_variables = len(sim_df.columns)

In [ ]:
counterfactual_df = base_df['label']

## Counterfactual generative adverserial network

* The generator consists of a transformer architecture adapted for continous time series simulations.
* The discriminator optimises for similarities in distributions across real and simulated data and corectness of the learnt distribution w.r.t. validation data.
* Conditional time series can be fed to the architecture to investigate counterfactual scenarios through simulation studies.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Input
from keras.optimizers import Adam
from sklearn.metrics import mean_squared_error

In [ ]:
gan_params = {
   'num_epochs': 1000,
   'save_interval': 100,
   'num_timesteps': 6,
   'num_variables': num_variables,
   'batch_size': 32,
   'lr': 0.01 
}

In [ ]:
generator_params = {
   'num_layers': 1,
   'd_model': 256,
   'dff': 512,
   'num_heads': 1,
   'dropout_rate': 0.01
}

In [ ]:
discriminator_params = {
   'bi_lstm_units': 64,
   'dropout_rate': 0.3,
   'lr': 0.01
}

In [ ]:
t_at_n = []
n_to_t = []
t_at_n_plus_one = []
n_plus_one_to_t_plus_one = []
cf_up_to_n = []
cf_up_to_n_plus_one = []

for i in range(int(num_obs / gan_params['num_timesteps'])):
    n_to_t.append(sim_df[i:(i+gan_params['num_timesteps'])].values)
    t_at_n.append(sim_df[(i+gan_params['num_timesteps']):(i+gan_params['num_timesteps']+1)].values)
    t_at_n_plus_one.append(sim_df[(i+gan_params['num_timesteps']+1):(i+gan_params['num_timesteps']+2)].values)
    n_plus_one_to_t_plus_one.append(sim_df[i+1:(i+gan_params['num_timesteps']+1)].values)
    cf_up_to_n.append(counterfactual_df[i:(i+gan_params['num_timesteps']+1)].values)
    cf_up_to_n_plus_one.append(counterfactual_df[i+1:(i+gan_params['num_timesteps']+2)].values)

In [ ]:
src_input = np.asarray(t_at_n)
src_input = src_input.reshape((int(num_obs / gan_params['num_timesteps']), num_variables, 1))

In [ ]:
src_input.shape

In [ ]:
tar_input = np.asarray(t_at_n_plus_one)
tar_input = tar_input.reshape((int(num_obs / gan_params['num_timesteps']), num_variables, 1))

In [ ]:
src_history_input = np.asarray(n_to_t)
src_history_input = src_history_input.reshape((int(num_obs / gan_params['num_timesteps']),
                                               num_variables, gan_params['num_timesteps']))

In [ ]:
src_history_input.shape

In [ ]:
tar_history_input = np.asarray(n_plus_one_to_t_plus_one)
tar_history_input = tar_history_input.reshape((int(num_obs / gan_params['num_timesteps']),
                                               num_variables, gan_params['num_timesteps']))

In [ ]:
cf_src_features = np.asarray(cf_up_to_n)
cf_src_features = cf_src_features.reshape((int(num_obs / gan_params['num_timesteps']), gan_params['num_variables'],
                                          gan_params['num_timesteps']+1))

In [ ]:
cf_tar_features = np.asarray(cf_up_to_n_plus_one)
cf_tar_features = cf_tar_features.reshape((int(num_obs / gan_params['num_timesteps']), gan_params['num_variables'],
                                          gan_params['num_timesteps']+1))

### Generator

The generator consists of transformer-based time series generator. A set of variables at fixed positions is used as source, and that same set of in the next timestep is used as target. Since the model is ran over continuous variables, rather than an embedding $n$ context timesteps are fed to the model encoded with relative positional embeddings. Since the autoregression of interest are learnt from the context timesteps rather than from the position of the variable in $\textbf{x}$ (where ordering is fixed but random), some modifications had to be made to ensure the model learnt correct dependencies between variables in the multivariate set $\textbf{x}$. Furthermore, rather than tokens, the generator needs to learn continous variables. The approach here is inspired by an application of the transformer architecture to music generation by [the Google Magenta team](https://magenta.tensorflow.org/music-transformer).

The Transformer implementation in TensorFlow Keras is adapted for timeseries from [the TensorFlow transformer tutorial](https://www.tensorflow.org/beta/tutorials/text/transformer) for neural machine translation.

In [ ]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

In [ ]:
def variable_encoding(variables, d_model):
    angle_rads = get_angles(np.arange(variables)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)
    sines = np.sin(angle_rads[:, 0::2])
    cosines = np.cos(angle_rads[:, 1::2])
    pos_encoding = np.concatenate([sines, cosines], axis=-1)
    pos_encoding = pos_encoding[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
encoding = variable_encoding(gan_params['num_variables'], generator_params['dff'])
print (encoding.shape)

plt.pcolormesh(encoding[0], cmap='RdBu')
plt.xlabel('Depth')
plt.xlim((0, generator_params['dff']))
plt.ylabel('Variable')
plt.colorbar()
plt.show()

In [ ]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

In [ ]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True) 
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
  
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % self.num_heads == 0
        self.depth = d_model // self.num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)
          
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
      
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
      
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
      
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size) 
        v = self.split_heads(v, batch_size) 

        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
    
        output = self.dense(concat_attention)
        return output, attention_weights

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([tf.keras.layers.Dense(dff, activation='relu'), tf.keras.layers.Dense(d_model)])

In [ ]:
class TimeseriesEncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, num_variables, num_timesteps, rate=0.1):
        super(TimeseriesEncoderLayer, self).__init__()
        
        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)
    
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
      
    def call(self, inputs, x, training, mask):
        attn_output, _ = self.mha(x, k=x, q=inputs, mask=mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

In [ ]:
sample_encoder = TimeseriesEncoderLayer(512, 8, 2048, gan_params['num_variables'], gan_params['num_timesteps'])
sample_encoder_output = sample_encoder(
    tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], 1)),
    x=tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], gan_params['num_timesteps'])),
    training=False, mask=None)
sample_encoder_output.shape  # (batch_size, input_seq_len, d_model)

In [ ]:
class TimeseriesDecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, num_variables, num_timesteps, rate=0.1):
        super(TimeseriesDecoderLayer, self).__init__()
    
        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)
    
        self.ffn = point_wise_feed_forward_network(d_model, dff)
     
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
      
      
    def call(self, inputs, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, k=x, q=inputs, mask=look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + inputs)
        
        attn2, attn_weights_block2 = self.mha2(enc_output, k=enc_output, q=out1, mask=padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)
        
        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)
        return out3, attn_weights_block1, attn_weights_block2

In [ ]:
sample_decoder = TimeseriesDecoderLayer(512, 8, 2048, gan_params['num_variables'], gan_params['num_timesteps'])

sample_decoder_output, _, _ = sample_decoder(
    tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], 1)),
    x=tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], gan_params['num_timesteps'])),
    enc_output=sample_encoder_output, 
    training=False,
    look_ahead_mask=None,
    padding_mask=None)

sample_decoder_output.shape

In [ ]:
class TimeseriesEncoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, num_variables, num_timesteps, rate=0.1):
        super(TimeseriesEncoder, self).__init__()
    
        self.d_model = d_model
        self.num_layers = num_layers
        self.num_variables = num_variables
        self.num_timesteps = num_timesteps
        
        self.variable_encoding = variable_encoding(num_variables, self.d_model)        
        self.enc_layers = [TimeseriesEncoderLayer(d_model, num_heads, dff,
                                                  num_variables, num_timesteps, rate) for _ in range(num_layers)]
      
        self.dropout = tf.keras.layers.Dropout(rate)
            
    def call(self, inputs, history, training, mask):
        x = tf.concat([inputs, history], axis=-1)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # x += self.variable_encoding[:, :self.num_variables, :]
        
        x = self.dropout(x, training=training)
        
        for i in range(self.num_layers):
            x = self.enc_layers[i](inputs, x=x, training=training, mask=mask)
        
        return x

In [ ]:
sample_encoder = TimeseriesEncoder(num_layers=2, d_model=generator_params['d_model'], num_heads=8, 
                                   dff=generator_params['dff'], num_variables=gan_params['num_variables'],
                                   num_timesteps=gan_params['num_timesteps'])

sample_encoder_output = sample_encoder(
    tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], 1)),
    history=tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], 
                               gan_params['num_timesteps'])),
    training=False, mask=None)
print (sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

In [ ]:
# print (sample_encoder_output)

In [ ]:
class TimeseriesDecoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, num_variables, num_timesteps, rate=0.1):
        super(TimeseriesDecoder, self).__init__()
    
        self.d_model = d_model
        self.num_layers = num_layers
        self.num_variables = num_variables
        self.num_timesteps = num_timesteps
        
        # self.variable_encoding = variabe_encoding(num_variables, d_model)
        self.dec_layers = [TimeseriesDecoderLayer(d_model, num_heads, dff, num_variables, num_timesteps, rate)
                           for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
      
    def call(self, tar, tar_hist, enc_output, training, look_ahead_mask, padding_mask):
        attention_weights = {}
        
        x = tf.concat([tar, tar_hist], axis=-1)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        # x += self.variable_encoding[:, :self.num_variables, :]
        
        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            #x, enc_output, training, look_ahead_mask, padding_mask
            x, block1, block2 = self.dec_layers[i](tar, x=x, enc_output=enc_output, training=training,
                                                   look_ahead_mask=look_ahead_mask, padding_mask=padding_mask)
            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
        
        return x, attention_weights

In [ ]:
sample_decoder = TimeseriesDecoder(num_layers=2, d_model=512, num_heads=8, dff=2048, 
                                   num_variables=gan_params['num_variables'],
                                   num_timesteps=gan_params['num_timesteps'])

output, attn = sample_decoder(
    tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], 1)),
    tar_hist=tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], 
                               gan_params['num_timesteps'])), 
    enc_output=sample_encoder_output, 
    training=False,
    look_ahead_mask=None, 
    padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape

In [ ]:
class TimeseriesTransformer(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, num_variables, num_timesteps, rate=0.1):
        super(TimeseriesTransformer, self).__init__()

        self.encoder = TimeseriesEncoder(num_layers, d_model, num_heads, dff, num_variables, num_timesteps, rate)
        self.decoder = TimeseriesDecoder(num_layers, d_model, num_heads, dff, num_variables, num_timesteps, rate)
        self.final_layer = tf.keras.layers.Dense(num_variables)
    
    def call(self, inputs, hist=None, tar=None, tar_hist=None, training=False, enc_padding_mask=None,
             look_ahead_mask=None, dec_padding_mask=None):
        enc_output = self.encoder(inputs, history=hist, training=training, mask=enc_padding_mask)
        dec_output, attention_weights = self.decoder(tar, tar_hist=hist, enc_output=enc_output, training=training,
                                                     look_ahead_mask=look_ahead_mask, padding_mask=dec_padding_mask)
    
        final_output = self.final_layer(dec_output)
        return final_output, attention_weights

In [ ]:
sample_transformer = TimeseriesTransformer(num_layers=2, d_model=512, num_heads=8, dff=2048, 
                                           num_timesteps=gan_params['num_timesteps'],
                                           num_variables=gan_params['num_variables'])

t = tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], 1))
hist = tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], gan_params['num_timesteps']))
t_plus_one = tf.random.uniform((gan_params['batch_size'], gan_params['num_variables'], 1))

fn_out, _ = sample_transformer(t, hist=hist, tar=t_plus_one)

fn_out.shape

### Build the transformer model for the generator

In [ ]:
def build_generator(params):
    t = tf.keras.layers.Input(shape=(params['num_variables'], 1))
    hist = tf.keras.layers.Input(shape=(params['num_variables'], params['num_timesteps']))
    tar = tf.keras.layers.Input(shape=(params['num_variables'], 1))
    tar_hist = tf.keras.layers.Input(shape=(params['num_variables'], params['num_timesteps']))
    
    transformer = TimeseriesTransformer(num_layers=2, d_model=params['d_model'], num_heads=params['num_heads'], 
                                        dff=params['dff'], num_variables=params['num_variables'], 
                                        num_timesteps=params['num_timesteps'])
    output, weights = transformer(t, hist=hist, tar=tar, tar_hist=tar_hist)
    g = tf.keras.layers.GaussianNoise(params['dropout_rate'])(output)
    g = tf.keras.layers.Dense(params['num_variables'], activation='sigmoid')(g)
    
    generator = tf.keras.models.Model(inputs=[t, hist, tar, tar_hist], outputs=g, name='generator')
    generator.summary()
    
    return generator

In [ ]:
generator = build_generator({**gan_params, **generator_params})

In [ ]:
generator.compile('adam', loss='mse', metrics=['mse'])

In [ ]:
generator.fit([src_input, src_history_input, tar_input, tar_history_input], epochs=5)

### Discriminator

In [ ]:
def build_discriminator(params):    
    output = tf.keras.layers.Input(shape=(params['num_variables']))
    
    d = tf.keras.layers.Dense(params['bi_lstm_units'], activation='tanh')(output)
    d = tf.keras.layers.Dropout(params['dropout_rate'])(d)
    d = tf.keras.layers.Dense(1, activation='softmax')(d)
    
    discriminator = tf.keras.models.Model(output, d, name="discriminator")
    discriminator.summary()
    
    return discriminator

In [ ]:
discriminator = build_discriminator({**gan_params, **discriminator_params})
discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

### CGAN

In [ ]:
def build_cgan(generator, discriminator, params):
    discriminator.trainable = False
    
    t = tf.keras.layers.Input(shape=(params['num_variables'], 1))
    hist = tf.keras.layers.Input(shape=(params['num_variables'], params['num_timesteps']))
    tar = tf.keras.layers.Input(shape=(params['num_variables'], 1))
    tar_hist = tf.keras.layers.Input(shape=(params['num_variables'], params['num_timesteps']))
    
    generated_variables = generator([t, hist, tar, tar_hist])
    classified = discriminator(generated_variables)
    
    cgan = tf.keras.models.Model([t, hist, tar, tar_hist], [generated_variables, classified], name='cgan')
    cgan.summary()
    return cgan

In [ ]:
losses = {
    "generator": "mean_squared_error",
    "discriminator": "binary_crossentropy"
}

In [ ]:
loss_weights = { "generator": 1.0, "discriminator": 0.5 }

In [ ]:
cgan = build_cgan(generator, discriminator, gan_params)
cgan.compile(loss=losses, loss_weights=loss_weights, optimizer='adam', metrics=['mse', 'accuracy'])

In [ ]:
def train_cgan(src, src_hist, tar, tar_hist, batch_size, params):
    g_metrics = []
    d_metrics = []
    
    half_batch = int(batch_size / 2)
    
    real_classified_as_real = np.ones(half_batch)
    synth_classified_as_fake = np.zeros(half_batch)
    discriminator_label_batch = np.concatenate((real_classified_as_real, synth_classified_as_fake))
    
    for i in range(params['num_epochs']):
        # create input of real and synthetic data
        random_index = np.random.randint(0, len(feature_data) - batch_size)
        src_batch = src[random_index:int(random_index + half_batch)]
        src_hist_batch = src_hist[random_index:int(random_index + half_batch)]
        synth_features = np.random.normal(-1.0, 1.0, size=[half_batch, params['num_timesteps'], 
                                                           feature_data.shape[2]])
        generator_input_batch = np.concatenate((real_features, synth_features))
        
        real_labels = label_data[random_index:int(random_index + half_batch)]
        synth_labels = np.random.normal(-1.0, 1.0, size=[half_batch, 1, feature_data.shape[2]])
        generator_label_batch = np.concatenate((real_labels, synth_labels))
        
        # apply generator
        generated = generator.predict(generator_input_batch)
        
        # train discriminator
        disc = discriminator.train_on_batch(generated)
                                                            
        # train gan
        gen_ = cgan.train_on_batch([generator_input_batch, ], [generator_label_batch, discriminator_label_batch])
        if i % 100 == 0:
            print('Epoch %s losses: discriminator %s, generator: %s' % (i, str(disc[0]), str(gen_[0])))
        
        d_metrics.append(disc)
        g_metrics.append(gen_)
    return d_metrics, g_metrics

In [ ]:
d_metrics, g_metrics = train_cgan(t_input, history_input, gan_params['batch_size'], gan_params)

In [ ]:
plt.figure(figsize=(15,7))
plt.plot([metrics[0] for metrics in d_metrics], label='discriminator loss')
plt.plot([metrics[0] for metrics in g_metrics], label='generator loss')
plt.legend()
plt.title('CGAN losses')
plt.show()

In [ ]:
plt.figure(figsize=(15,7))
plt.plot([metrics[1] for metrics in d_metrics], label='discriminator mean squared error')
plt.plot([metrics[1] for metrics in g_metrics], label='generator mean average error')
plt.legend()
plt.title('CGAN performance metrics')
plt.show()

In [ ]:
generated_y = generator.predict(np.random.rand(num_obs, gan_params['sequence_length'], gan_cols))[:,-1,-1]
gan_y = gan_df['label'].values

In [ ]:
plt.figure(figsize=(15,7))
plt.plot(gan_y, label='observed cpi')
plt.plot(generated_y, label='gan-generated cpi')
plt.legend()
plt.title('Observed versus GAN-generated values for consumer price inflation in %s' % country)
plt.show()

In [ ]:
print('rmse: %s\nmean observed: %s\nmean generated: %s' % (np.sqrt(mean_squared_error(gan_y, generated_y)),
                                                           np.mean(gan_y), np.mean(generated_y)))